# Building Similar Item Recommendation

## Imports 

Python ships with a broad collection of libraries and we need to import those as well as the ones installed to help us like boto3(The AWS SDK) and Pandas/Numpy which are core data science tools

In [1]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time


Next you will want to validate that your environment can communicate successfully with Amazon Personalize, the lines below do just that.

In [2]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

In [3]:
%store -r dataset_group_arn
%store -r GetRecommendationsByItem_arn
%store -r useritems
%store -r bucket

no stored variable bucket


## Create the Solution and Version

In Amazon Personalize a trained model is called a Solution, each Solution can have many specific versions that relate to a given volume of data when the model was trained.

To begin we will list all the recipies that are supported, a recipie is an algorithm that has not been trained on your data yet. After listing you'll select one and use that to build your model.

### Select Recipe

In [4]:
recipe_arn = "arn:aws:personalize:::recipe/aws-sims" 
# SIMS selected for this solution.

### Create and Wait for Solution

First you will create the solution with the API, then you will create a version. It will take several minutes to train the model and thus create your version of a solution. Once it gets started and you are seeing the in progress notifications it is a good time to take a break, grab a coffee, etc.

#### Create Solution

In [5]:
create_solution_response = personalize.create_solution(
    name = "personalize-demo-soln-sims",
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn
)

sims_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:962222257213:solution/personalize-demo-soln-sims",
  "ResponseMetadata": {
    "RequestId": "60b5ef84-1ed6-4fba-a796-e4ed59075573",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 25 Feb 2020 05:09:36 GMT",
      "x-amzn-requestid": "60b5ef84-1ed6-4fba-a796-e4ed59075573",
      "content-length": "96",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Create Solution Version

In [6]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = sims_solution_arn
)

sims_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:962222257213:solution/personalize-demo-soln-sims/82ab4692",
  "ResponseMetadata": {
    "RequestId": "6539d4eb-cba7-4b55-986d-2f981331a91b",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 25 Feb 2020 05:09:36 GMT",
      "x-amzn-requestid": "6539d4eb-cba7-4b55-986d-2f981331a91b",
      "content-length": "112",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Wait for Solution Version to Have ACTIVE Status

This will take at least 20 minutes.

In [7]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = sims_solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

SolutionVersion: CREATE PENDING
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGR

#### Get Metrics of Solution Version

Now that your solution and version exists, you can obtain the metrics for it to judge its performance. These metrics are not particularly good as it is a demo set of data, but with larger more compelx datasets you should see improvements.

In [8]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = sims_solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:962222257213:solution/personalize-demo-soln-sims/82ab4692",
  "metrics": {
    "coverage": 0.0832,
    "mean_reciprocal_rank_at_25": 0.0195,
    "normalized_discounted_cumulative_gain_at_10": 0.0239,
    "normalized_discounted_cumulative_gain_at_25": 0.0261,
    "normalized_discounted_cumulative_gain_at_5": 0.0225,
    "precision_at_10": 0.003,
    "precision_at_25": 0.0015,
    "precision_at_5": 0.0051
  },
  "ResponseMetadata": {
    "RequestId": "05af6ded-2de7-4498-b96d-dee9f7da4641",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 25 Feb 2020 05:48:39 GMT",
      "x-amzn-requestid": "05af6ded-2de7-4498-b96d-dee9f7da4641",
      "content-length": "409",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


## Create and Wait for the Campaign

Now that you have a working solution version you will need to create a campaign to use it with your applications. A campaign is simply a hosted copy of your model. Again there will be a short wait so after executing you can take a quick break while the infrastructure is being provisioned.

#### Create Campaign

In [9]:
create_campaign_response = personalize.create_campaign(
    name = "personalize-camp-sims",
    solutionVersionArn = sims_solution_version_arn,
    minProvisionedTPS = 1
)

sims_campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:962222257213:campaign/personalize-camp-sims",
  "ResponseMetadata": {
    "RequestId": "b2790589-b39c-4769-96e5-b9aecad328d2",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 25 Feb 2020 05:48:39 GMT",
      "x-amzn-requestid": "b2790589-b39c-4769-96e5-b9aecad328d2",
      "content-length": "91",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Wait for Campaign to Have ACTIVE Status

In [10]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = sims_campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

Campaign: CREATE PENDING
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: ACTIVE


## Get Sample Recommendations

After the campaign is active you are ready to get recommendations. First we need to select a random user from the collection. Then we will create a few helper functions for getting the Allstore merchandise (item descriptions) information to show for recommendations instead of just IDs.

In [11]:
# Getting a random user:
user_id, item_id, _ = useritems.sample().values[0]
print("Item: {}".format(item_id))

Item: B000A0REKO


In [14]:
# First load items into memory
%store -r bucket
allitemuri = f's3://{bucket}/items_w_metadata.csv'
items = pd.read_csv(allitemuri, sep=',', usecols=[0,1], names=['asin', 'title'], index_col='asin',header=0)

# print(items)

def get_allstore_products(asin):
    """
    Takes in an ID, returns a title
    """
    asin = str(asin)
    return items.loc[asin]['title']

#### Call GetRecommendations

Using the user that you obtained above, the lines below will get recommendations for you and return the list of movies that are recommended.


In [15]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = sims_campaign_arn,
    itemId = str(item_id),
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("SIMS for item_id: ", item_id)

item_list = get_recommendations_response['itemList']

recommendation_list = []

for item in item_list:
    try :
        title = get_allstore_products(item['itemId'])
    except:
        continue
    recommendation_list.append(title)  
    
recommendations_df = pd.DataFrame(recommendation_list, columns = ['Item Description'])
recommendations_df

SIMS for item_id:  B000A0REKO


,Item Description
0,Butler Creek Lula Universal Pistol Magazine Lo...
1,Howard Leight R-01526 Impact Sport Electronic ...
2,Rothco 550lb. Type III Nylon Paracord
3,Hoppe's BoreSnake Rifle Bore Cleaner (Choose Y...
4,Magnesium Fire Starter
5,Emergency Mylar Thermal Blankets (Pack of 10)
6,OFF SET TACTICAL Weapon Mount For Olight M20/M...
7,Butler Creek LULA All-in-One Magazine Speed Lo...
8,Morakniv Companion Fixed Blade Outdoor Knife w...
9,Ultralight Backpacking Canister Camp Stove wit...


## Update Lambda Configuration

In [16]:
lambdaclient = boto3.client('lambda')

response = lambdaclient.get_function(
    FunctionName=GetRecommendationsByItem_arn,
)

Environment = response['Configuration']['Environment']
print(Environment)

#update Env var

Environment['Variables']['Campaign_ARN'] = sims_campaign_arn

response = lambdaclient.update_function_configuration(
    FunctionName=GetRecommendationsByItem_arn,
    Environment=Environment
)

{'Variables': {'Campaign_ARN': 'arn:aws:personalize:us-east-1:387269085412:campaign/SIM-campaign', 'ddb_tablename': 'teststr-Items'}}


## Review

Using the codes above you have successfully trained a deep learning model to generate Allstore Merchandise recommendations based on prior user behavior. Think about other types of problems where this data is available and what it might look like to build a system like this to offer those recommendations.

Now you are ready to move onto the next notebook `3.Building_Campaign_P-rank.ipynb`



## Notes for the Next Notebook:

There are a few values you will need for the next notebook, execute the cells below to store them so they can be copied and pasted into the next part of the exercise.